In [1]:

# Default locations if parameter not passed
benchmark_data_path = "/scratch/rodrigo.freitas/remy-schedulers/tb-compare.pkl"


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
from rich import print as rprint
import seaborn as sns
import os
import re
from scipy.stats import bootstrap

benchmark_data = pickle.load(open(benchmark_data_path, "rb"))

df = pd.DataFrame(benchmark_data["dataframe"])

sns.set_theme()

colors = [
    "#7f7f7f",  # grey
    "#621dac",  # main purple
    "#c5702d",  # orange
    "#000000",  # black,
    "#099892",  # teal
    "#ffd400",  # yellow
    "#7e57c4",  # pink/purple,
]
colors_backup = colors.copy()

def calculate_bootstrap_error(data):
    n_runs = benchmark_data["config"].metadata.runs
    if n_runs < 2:
        return 0
    res = bootstrap(
        (data,), np.mean, confidence_level=0.95, n_resamples=1000, method="basic"
    )
    return res.standard_error

In [3]:
bench_names = df["name"].unique()
print(f"Benchmarks found: {bench_names}")
run_metadata = benchmark_data["config"].applications
n_runs = benchmark_data["config"].metadata.runs

df_bench = df[df["name"] == 'tb-compare']


print(df_bench.columns)
print(df_bench.head())


Benchmarks found: ['tb-compare']
Index(['name', 'node_count', 'iter', 'image', 'full_output', 'hosts',
       'ompc_tb_iter_num', 'type', 'tb_path', 'time'],
      dtype='object')
         name  node_count   iter               image  \
0  tb-compare           1   1000  scheduler-main.sif   
1  tb-compare           1   1000  scheduler-main.sif   
2  tb-compare           1  10000  scheduler-main.sif   
3  tb-compare           1  10000  scheduler-main.sif   
4  tb-compare           2   1000  scheduler-main.sif   

                                         full_output  \
0  [1749737225.509686] [sorgan-cpu1:321788:0]    ...   
1  [1749737226.241517] [sorgan-cpu1:321902:0]    ...   
2  [1749737226.965344] [sorgan-cpu1:322019:0]    ...   
3  [1749737227.692036] [sorgan-cpu1:322137:0]    ...   
4  [1749737228.422309] [sorgan-cpu1:322254:0]    ...   

                                             hosts  ompc_tb_iter_num type  \
0  sorgan-cpu1,sorgan-cpu2,sorgan-cpu3,sorgan-cpu4                10 

In [4]:
# print the field full_output of the first row
print("Full output of the first row:")

# tb_time field (seconds)
# get total time from the output
def extract_elapsed_time(text):
    matches = re.findall(r"Elapsed Time ([\d.eE+-]+) seconds", text)
    if matches:
        # gets last mach
        # float() already handles scientific notation
        return float(matches[-1])
    return float('nan')

def extract_total_sched_time(text):
    # Find all "Scheduling : <number>"
    matches = re.findall(r"Scheduling\s*:\s*(\d+)", text)
    if not matches:
        return None
    total_microseconds = sum(int(m) for m in matches)
    return total_microseconds / 1e6  # microseconds to seconds

df_bench['tb_time'] = df_bench['full_output'].apply(extract_elapsed_time)
df_bench['sched_time'] = df_bench['full_output'].apply(extract_total_sched_time)

print(df_bench.columns)



Full output of the first row:
Index(['name', 'node_count', 'iter', 'image', 'full_output', 'hosts',
       'ompc_tb_iter_num', 'type', 'tb_path', 'time', 'tb_time', 'sched_time'],
      dtype='object')


In [5]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# 1) bootstrap CI
def bootstrap_ci(series, n_bootstrap=1000, ci=95):
    boot_means = np.random.choice(series, (n_bootstrap, len(series)), replace=True).mean(axis=1)
    lower = np.percentile(boot_means, (100 - ci) / 2)
    upper = np.percentile(boot_means, 100 - (100 - ci) / 2)
    return pd.Series({'mean': series.mean(), 'ci_lower': lower, 'ci_upper': upper})

# 2) group and apply
group_fields = ['name', 'node_count', 'iter', 'ompc_tb_iter_num', 'image', 'type', 'tb_path']
float_fields = ['tb_time', 'sched_time']

grouped = (
    df_bench
    .groupby(group_fields)[float_fields]
    .apply(lambda df: df.apply(bootstrap_ci))
    .reset_index()
)

# 3) correct flatten
grouped.columns = [
    f"{c[0]}_{c[1]}" if isinstance(c, tuple) else c
    for c in grouped.columns
]

# sanity check
print(grouped.columns.tolist())

# 4) plotting function
def plot_ci_line(data, x, y_mean, y_ci_lower, y_ci_upper, label):
    sns.lineplot(data=data, x=x, y=y_mean, label=label)
    plt.fill_between(
        data[x],
        data[y_ci_lower],
        data[y_ci_upper],
        alpha=0.3
    )

# 5) draw
for metric in ['tb_time', 'sched_time']:
    plt.figure()
    plot_ci_line(
        grouped,
        x='node_count',
        y_mean=f'{metric}_mean',
        y_ci_lower=f'{metric}_ci_lower',
        y_ci_upper=f'{metric}_ci_upper',
        label=metric
    )
    plt.title(f'{metric} vs node_count (95% bootstrap CI)')
    plt.xlabel('node_count')
    plt.ylabel(f'{metric} (seconds)')
    plt.grid(True)
    plt.legend()
    plt.savefig(f'plots/{metric}_vs_node_count.pdf')
    plt.show()


['name', 'node_count', 'iter', 'ompc_tb_iter_num', 'image', 'type', 'tb_path', 'level_7', 'tb_time', 'sched_time']


ValueError: Could not interpret value `tb_time_mean` for `y`. An entry with this name does not appear in `data`.

<Figure size 640x480 with 0 Axes>